In [14]:
import os
import pickle
import tsplib95
import networkx as nx
from networkx.algorithms.approximation.traveling_salesman import *
import pandas as pd
import numpy as np

In [ ]:
# load optimal tour lengths
opt_tour_lengths = dict()
with open("tsp_dataset/tsp.soln") as file:
    for line in file:
        name, _, opt_tour, *_ = line.split()
        opt_tour_lengths[name] = int(opt_tour)

def get_opt_tour_length(name):
    return opt_tour_lengths[name]

In [39]:
def get_tsp_graph(name) -> nx.Graph:
    # load from pkl file if exists
    graph_file_path = os.path.join("tsp_dataset", f"{name}.graph.pkl")
    if os.path.exists(graph_file_path):
        # print(f"Loading {name} from {graph_file_path}")
        with open(graph_file_path, "rb") as file:
            G = pickle.load(file)
        return G

    tsp_file_path = os.path.join("tsp_dataset", f"{name}.tsp")
    print(f"Loading {name} from {tsp_file_path}")
    problem = tsplib95.load(tsp_file_path)
    G = problem.get_graph()

    if problem.edge_weight_type == "EXPLICIT" and problem.edge_weight_format == "FULL_MATRIX":
        for u in G.nodes:
            for v in G.nodes:
                if u > v:
                    G.remove_edge(u, v)

    loop_edges = list(nx.selfloop_edges(G))
    G.remove_edges_from(loop_edges)

    n, m = G.number_of_nodes(), G.number_of_edges()
    assert n * (n - 1) // 2 == m

    # save to pkl file
    with open(graph_file_path, "wb") as file:
        pickle.dump(G, file)

    return G

In [16]:
data = []

for tsp_file in os.listdir("tsp_dataset"):
    name, file_extn = os.path.splitext(tsp_file)
    if file_extn == ".tsp":
        tsp_file_path = os.path.join("tsp_dataset", f"{name}.tsp")
        problem = tsplib95.load(tsp_file_path)
        data.append({
            "name": name,
            "dimension": problem.dimension,
            "optimal_tour_length": get_opt_tour_length(name),
            "type": problem.type,
            "comment": problem.comment,
            "edge_weight_type": problem.edge_weight_type,
            "edge_weight_format": problem.edge_weight_format,
        })

In [25]:
df = pd.DataFrame(data)

In [26]:
# sort df by dimension
df = df.sort_values(by="dimension").reset_index(drop=True)

In [33]:
# print all dimensions
print(df["dimension"].unique())

[  14   16   17   21   22   24   26   29   42   48   51   52   58   70
   76   96   99  100  101  105  107  120  124  127  130  136  137  144
  150  152  159  175  195  198  200  202  225  226  229  262  264  280
  299  318  400  417  431  439  442  493  535  561  574  575  654  657
  666  724  783 1000 1002 1032 1060 1084 1173 1291 1304 1323 1379 1400
 1432 1655 1748 1817 1889 2152 2319 2392 3038 4461 7397]


In [ ]:
def get_tour_length(G: nx.graph, tour: list):
    tour_length = 0
    for i in range(len(tour) - 1):
        u, v = tour[i], tour[i + 1]
        w = G[u][v]["weight"]
        tour_length += w
    return tour_length

In [34]:
df.head()

,name,dimension,optimal_tour_length,type,comment,edge_weight_type,edge_weight_format
0,burma14,14,3323,TSP,14-Staedte in Burma (Zaw Win),GEO,FUNCTION
1,ulysses16,16,6859,TSP,Odyssey of Ulysses (Groetschel/Padberg),GEO,None
2,gr17,17,2085,TSP,17-city problem (Groetschel),EXPLICIT,LOWER_DIAG_ROW
3,gr21,21,2707,TSP,21-city problem (Groetschel),EXPLICIT,LOWER_DIAG_ROW
4,ulysses22,22,7013,TSP,Odyssey of Ulysses (Groetschel/Padberg),GEO,None


In [43]:
for row in df.itertuples():
    name = row.name
    print(name)

    G = get_tsp_graph(name)
    G = G.to_undirected()
    
    tour = christofides(G)
    # print("tour:", tour)
    
    tsp_file_path = os.path.join("tsp_dataset", f"{name}.tsp")
    problem = tsplib95.load(tsp_file_path)
    tour_length = problem.trace_tours([tour])[0]
    
    apprx_ratio = tour_length / get_opt_tour_length(name)
    
    print(name, tour_length, apprx_ratio)
    
    # add tour_length and apprx_ratio to row
    df.at[row.Index, "tour_length"] = tour_length
    df.at[row.Index, "apprx_ratio"] = apprx_ratio
    print()
    # break

burma14
burma14 3607 1.0854649413180861

ulysses16
ulysses16 6984 1.0182242309374545

gr17
gr17 2197 1.0537170263788969

gr21
gr21 3092 1.1422238640561506

ulysses22
ulysses22 7412 1.0568943390845573

gr24
gr24 1455 1.1438679245283019

fri26
fri26 1043 1.1131270010672358

bayg29
bayg29 1716 1.0658385093167702

bays29
bays29 2155 1.066831683168317

swiss42
swiss42 1393 1.0942655145326001

dantzig42
dantzig42 761 1.0886981402002862

gr48
gr48 5753 1.140110978993262

hk48
hk48 12896 1.1252072245004798

eil51
eil51 462 1.0845070422535212

berlin52
berlin52 8560 1.1349774595597986

brazil58
brazil58 27442 1.080606418586336

st70
st70 771 1.1422222222222222

eil76
eil76 608 1.1301115241635689

pr76
pr76 116684 1.0788191458870737

gr96
gr96 61952 1.1221358836421598

rat99
rat99 1393 1.1502890173410405

kroC100
kroC100 22752 1.0965347727601331

kroD100
kroD100 23781 1.116793462947309

kroE100
kroE100 23819 1.0793456588725756

rd100
rd100 8906 1.1259165613147915

kroB100
kroB100 24012 1.0845038

KeyboardInterrupt: 

In [ ]:
# save df
df.to_csv("tsp_dataset/tsp_results.csv", index=False)

In [ ]:
problem = tsplib95.load("tsp_dataset/a280.tsp")
opt_tour_length = 2020

In [ ]:
problem.as_dict()

In [ ]:
G = problem.get_graph()

In [ ]:
G.graph

In [ ]:
G.number_of_nodes(), G.number_of_edges()

In [ ]:
list(G.edges(data=True))

In [ ]:
G.number_of_nodes(), G.number_of_edges()

In [ ]:
n, m = G.number_of_nodes(), G.number_of_edges()
print(n, m)

In [ ]:
assert n * (n - 1) // 2 == m

In [ ]:
tour = christofides(G)
print("tour:", tour)
tour_length = problem.trace_tours([tour])[0]
print("tour length:", tour_length)
print("approximation ratio:", tour_length / opt_tour_length)